In [3]:
# Imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
from sklearn import svm
from skimage.feature import greycomatrix, greycoprops
from sklearn import metrics
from skimage.feature import hog

## Read Data

In [6]:

x_data, y_data = [], []
#load male images
for filename in sorted(glob.glob('CMP23-Dataset/Male/*.jpg')):
    try:
        img = cv2.imread(filename, 0)                           # 0 for gray-scale
        x_data.append(img)
        y_data.append(0)
    except:
        print("corrupted image detected.")
    

## load female images
for filename in sorted(glob.glob('CMP23-Dataset/Female/*.jpg')):
    try:
        img = cv2.imread(filename, 0)
        x_data.append(img)
        y_data.append(1)
    except:
        print("corrupted image detected.")
    

x_data = np.array(x_data)
y_data = np.array(y_data)

# Shuffle both
p = np.random.permutation(len(x_data))
x_data = x_data[p]
y_data = y_data[p]

train_size = int(0.8 * len(x_data))
x_train, y_train = x_data[:train_size], y_data[:train_size]
x_test, y_test = x_data[train_size:], y_data[train_size:]



/tmp/ipykernel_93356/3991342258.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_data = np.array(x_data)


## Extract Features

In [7]:
def get_hog_feature(imag):
    fd, hog_image = hog(imag, orientations=8, pixels_per_cell=(16, 16),
                        cells_per_block=(1, 1), visualize=True)
                      
    return fd

In [ ]:
def get_glcm_feature(gray_scale_img, name):
    GLCM = greycomatrix(gray_scale_img, distances = [1], angles = [0, np.pi/2, np.pi, np.pi * 1.5 ], levels=256, normed=True)
    # P[i,j,d,theta] is the number of times that grey-level j occurs at a distance d and at an angle theta from grey-level i.
    return np.sum(np.abs(greycoprops(GLCM, prop=name)))
    

In [ ]:
# converting the dataset into features (x1, x2)
x_train = np.array([(get_glcm_feature(img, 'energy'), (get_glcm_feature(img, 'homogeneity'))) for img in x_train])
x_test = np.array([(get_glcm_feature(img, 'energy'), (get_glcm_feature(img, 'homogeneity'))) for img in x_test])

In [8]:

x_train_hog = np.array([(get_hog_feature(img), (get_hog_feature(img))) for img in x_train])
x_test_hog = np.array([(get_hog_feature(img), (get_hog_feature(img))) for img in x_test])

In [ ]:
# Generate scatter plot for training data 
x_train_male_hog = x_train_hog[y_train == 0]
x_train_female_hog =x_train_hog[y_train == 1]
plt.scatter(x_train_male_hog[:,0], x_train_male_hog[:,1], color='red')
plt.scatter(x_train_female_hog[:,0], x_train_female_hog[:,1], color='blue')
plt.title('Linearly separable data')
plt.xlabel('feature 1')
plt.ylabel('feature 2')
plt.show()


In [ ]:
# Generate scatter plot for training data 
x_train_male = x_train[y_train == 0]
x_train_female = x_train[y_train == 1]
plt.scatter(x_train_male[:,0], x_train_male[:,1], color='red')
plt.scatter(x_train_female[:,0], x_train_female[:,1], color='blue')
plt.title('Linearly separable data')
plt.xlabel('feature 1')
plt.ylabel('feature 2')
plt.show()
